## Trigger test with simulated data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.signal as ss

In [ ]:
def Signal(x, t_rise, t_decay, amplitude, t_arrival):
    '''This funtion generates a signal of the type we expect.'''
    return amplitude*(np.exp(-(x-t_arrival)/t_decay)-np.exp(-(x-t_arrival)/t_rise))

def Noise(sigma):
    '''This funtion creates some noise with a normal distribution with mean 0 and standard deviation "sigma".'''
    return np.random.normal(0, sigma)

def CreateSignal(t, t_arrival):
    '''Create a signal with the information contained in "signal_info". 
    It returns a double-exp signal 1000 points long and s.t. it's only positive.'''

    # -- Characteristics of the signal --
    signal_info = np.empty(4)
    signal_info[0] = np.random.uniform(3, 6) # Random value of the rise-time
    signal_info[1] = np.random.uniform(45, 60) # Random value of the decay-time
    signal_info[2] = np.random.uniform(0.6, 5) # Random value of the amplitude
    signal_info[3] = t_arrival # Time of arrival of the signal

    # Now we generate the shitty funtion with domain (0, +inf) that goes to -inf as x goes to zero
    single_signal = Signal(t, signal_info[0], signal_info[1], signal_info[2], signal_info[3])

    # Now we limit each signal such that it's only positive and 1000 points long
    single_signal = single_signal[int(signal_info[3]):int(signal_info[3])+1000]

    return single_signal, signal_info

def SignalSimulation(n_value, prob): 
    '''This function creates a complete set of signal of the type we expect and returns also how many events we generated'''
    clean_data = np.zeros(int(n_value)) # Vector that will contain the digital clean data we will obtain
    definitive_signal = np.empty(int(n_value))  # Vector that will contain the effective digital data (with noise) we will obtain
    t = np.linspace(0, int(n_value), int(n_value)) # Vector with the "time" value

    sum = 0 # We will use this variable to have a better estimation of the sigma of the noise
    n_signals = 0 # This will contain the number of signals that we will have
    i = 0

    # Cycle on clean_data that simulates the digital clean data we will obtain
    while i < len(clean_data):
        prob_signal = np.random.uniform(0,1) # We use this to have the probability of obtaining a signal

        # We generate a signal with a certain probability "prob"
        if (prob_signal < prob) and (i != int(n_value-1)): # Case in which we have the signal
            j = i

            # -- Generation of the signal --
            single_signal, signal_info = CreateSignal(t, j)
            sum = sum + signal_info[2]
            n_signals = n_signals + 1

            # Indexes to be removed to insert the signal
            removed_index = []
            k = i
            for l in range(0, 1000):
                removed_index.insert(l, k) # In position l of the removed_index vector we insert the index i of the clean_data vector
                k = k+1

            # We check if the index removing process we exceed the length of the original array
            if any( num > int(n_value) for num in removed_index):
                l = removed_index[-1] - int(n_value) # Number of indexes that exceeds the length of the original array
                del removed_index[-l-1:-1]
                del removed_index[-1]

                clean_data = np.delete(clean_data, removed_index)
                clean_data = np.insert(clean_data, j, single_signal[:int(len(removed_index))])

            else:
                clean_data = np.delete(clean_data, removed_index)
                clean_data = np.insert(clean_data, j, single_signal)

            if j+1000 < int(n_value):
                i = i + 1000
                j = j + 1000
            else: 
                break

        else:
            clean_data[i] = 0
            i = i + 1

    try:
        mean_amplitude = sum/n_signals
        sigma = 0.3*mean_amplitude # Sigma of the noise we are going to add

    except:
        sigma = 0.3 # We decide that this the value of the sigma of the noise if we don't have any signal

    # Now we add the noise and plot the signal
    for i in range(len(clean_data)):
        definitive_signal[i] = clean_data[i] + Noise(sigma) # We should define better the sigma of the noise but whatever
    
    return definitive_signal, n_signals

def Filter(signal, points = 101, approx = 5):
    '''This function filters a signal with the Savitzky-Golay filter'''

    filt_signal = ss.savgol_filter(signal, points, approx, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)

    return filt_signal

def AmplitudeTrigger(definitive_signal, filt_signal, length, trigger_level):
    '''This function uses a trigger on the amplitude of the signal and checks how many events we register'''
    signals = np.empty((0, length)) # Empty matrix that will contain the values that we are going to save
    i = 0
    n_signal = 0 # Number of the signal we are detecting

    while i < len(filt_signal):
    
        if filt_signal[i] > trigger_level:
            n_signal = n_signal + 1
            added_value = np.zeros(length)

            for j in range(0, length):
                try: 
                    # Case in which the signal is ok
                    added_value[j] = definitive_signal[i-100+j]
                except:
                    # Case when the signal is too long
                    added_value[j] = 0

            signals = np.insert(signals, 0, added_value, axis=0)

            i = i + 700
    
        else:
            i = i + 1

    return n_signal

In [ ]:
'''t = np.linspace(0, 10000, 10000) # It simulates the various time instants of the signal we will observe
n_events = int(np.random.uniform(0, 6)) # Number of events to be generated
print("We expect " + str(n_events) + " events.")

# We use a try-except because in the caso of zero signals there are some division by zero that will cause problem if not taken care of

try:

# ---------------- Characterization of the signals ------------------- #
    signal_info = np.empty((n_events, 4)) # Vector that will contain the rise-time, decay-time, amplitude and time of arrival of each signal
    sum = 0 # Variable that we will need after to do something

    for i in range(0, n_events):
        signal_info[i, 0] = np.random.uniform(3, 6) # Random value of the rise-time
        signal_info[i, 1] = np.random.uniform(45, 60) # Random value of the decay-time
        signal_info[i, 2] = np.random.uniform(0.6, 2) # Random value of the amplitude

        sum = sum + signal_info[i, 2]

    mean_amplitude = sum/n_events # Mean amplitude of the signals
    sigma = 0.68*0.3*mean_amplitude # Sigma of the noise

    print("\nWe have a mean amplitude of the signals of: " +  str(mean_amplitude) + ".")
    print("We have a sigma for the noise of: " +  str(sigma) + ".\n")

    signal_info[0, 3] = np.random.uniform(0, 0.025*max(t)) # Random value of the arrival time of the first signal

    for i in range(1, n_events):
        # Now we create the others time of arrival with each signal that can arrive between 1000 and 2000 points after the previous
        signal_info[i, 3] =  np.random.uniform(signal_info[i-1, 3] + 1000, signal_info[i-1, 3] + 2000)

    print("\n", signal_info, "\n")
# -------------------------------------------------------------------- #

# ----------------- Generation of the signals ------------------------ #

# Each signal is 1000 points long and will be put in the matrix 'events'

    events = np.empty((n_events, 1000)) # Matrix that will contain the signals values

    for i in range(0, n_events):
        # Now we generate the shitty funtion with domain (0, +inf) that goes to -inf as x goes to zero
        single_signal = Signal(t, signal_info[i, 0], signal_info[i, 1], signal_info[i, 2], signal_info[i, 3]) 
        
        signal_info[i, 3] = signal_info[i, 3] + 1
        
        # Now we limit each signal such that it's only positive and 1000 points long
        single_signal = single_signal[int(signal_info[i, 3]):int(signal_info[i, 3])+1000]
        events[i] = single_signal # We put the signal values into the matrix

        print("The index of arrival of the single signal number " + str(i+1) + " is " + str(int(signal_info[i, 3])))

# Now we start to create the "true" signal by adding the single signals together in a single vector
    total_signal = events[0]
    if n_events != 1:
        for i in range(1, n_events):
            total_signal = np.append(total_signal, events[i])

    plt.plot(t[:n_events*1000], total_signal)
    plt.title("Clean signals one after the other")
    plt.show()

# Now we add the space that will be necessary for the noise before the first signal

    initial_noise = np.zeros(len(t[:int(signal_info[0, 3])]))
    total_noise = initial_noise
    total_signals = np.insert(total_signal, 0, total_noise)

# Now we add the space of the noise before the remaining signals

    if n_events != 1:
        for i in range(1, n_events):
            initial_noise = np.zeros(int(signal_info[i, 3]) - int(signal_info[i-1, 3] + 1000))
            total_signal = np.insert(total_signal, int(signal_info[i-1, 3] + 1000), initial_noise)
            total_noise = np.append(total_noise, initial_noise)

    if len(total_signal) < len(t):
        final_noise = np.zeros( len(t) - len(total_signal) )
        total_signal = np.append( total_signal, final_noise )

    plt.plot(t, total_signal[:len(t)])
    plt.title("Complete clean plot of the signals")
    plt.show()

# Now we have to add the noise

    for i in range(len(t)):
        total_signal[i] = total_signal[i] + Noise(sigma)

    plt.plot(t, total_signal[:len(t)])
    plt.title("Signal vs time")
    plt.show()

except: 
    print("Since we expect zero events we have a vector with only noise")
    for i in range(len(t)):
       total_signal[i] = Noise(0.3)

    plt.plot(t, total_signal)
    plt.show()'''

# Simulation of the signals

My idea is to create a vector full of zeros and sometimes generate a signal (with a certain probability to be defined).

In [ ]:
n_value = 1e4 # Number of digital data we will obtain
prob = 0.0005 # Probability of observing a signal

clean_data = np.zeros(int(n_value)) # Vector that will contain the digital clean data we will obtain
definitive_signal = np.empty(int(n_value))  # Vector that will contain the effective digital data (with noise) we will obtain
t = np.linspace(0, int(n_value), int(n_value)) # Vector with the "time" value

sum = 0 # We will use this variable to have a better estimation of the sigma of the noise
n_signals = 0 # This will contain the number of signals that we will have
i = 0

# Cycle on clean_data that simulates the digital clean data we will obtain
while i < len(clean_data):
    prob_signal = np.random.uniform(0,1) # We use this to have the probability of obtaining a signal

    # We generate a signal with a certain probability "prob"
    if (prob_signal < prob) and (i != int(n_value)): # Case in which we have the signal
        j = i
        print("We have a signal at point: " + str(i))

        # -- Generation of the signal --
        single_signal, signal_info = CreateSignal(t, j)
        sum = sum + signal_info[2]
        n_signals = n_signals + 1

        # Indexes to be removed to insert the signal
        removed_index = []
        k = i
        for l in range(0, 1000):
            removed_index.insert(l, k) # In position l of the removed_index vector we insert the index i of the clean_data vector
            k = k+1

        # We check if the index removing process we exceed the length of the original array
        if any( num > int(n_value) for num in removed_index):
            l = removed_index[-1] - int(n_value) # Number of indexes that exceeds the length of the original array
            print("We are going to remove the last " + str(l) + " indexes")
            del removed_index[-l-1:-1]
            del removed_index[-1]

            print(removed_index)

            clean_data = np.delete(clean_data, removed_index)
            clean_data = np.insert(clean_data, j, single_signal[:int(len(removed_index))])

        else:
            clean_data = np.delete(clean_data, removed_index)
            clean_data = np.insert(clean_data, j, single_signal)

        if j+1000 < int(n_value):
            i = i + 1000
            j = j + 1000
        else: 
            break

    else:
        clean_data[i] = 0
        i = i + 1

# Now we shold have a clean plot with some signals
plt.plot(t, clean_data[:len(t)])
plt.show()

try:
    mean_amplitude = sum/n_signals
    sigma = 0.3*mean_amplitude # Sigma of the noise we are going to add

except:
    sigma = 0.3 # We decide that this the value of the sigma of the noise if we don't have any signal

# Now we add the noise and plot the signal
for i in range(len(clean_data)):
    definitive_signal[i] = clean_data[i] + Noise(sigma) # We should define better the sigma of the noise but whatever

plt.plot(t, definitive_signal[:len(t)])
plt.show()

Now we try the Savitzky-Golay filter

In [ ]:
filt_signal = ss.savgol_filter(definitive_signal, 101, 5, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)
plt.plot(t, filt_signal[:len(t)])

# Trigger of the signal

We start by doing a simple trigger on the amplitude. The idea is to cycle on the signal filtered by the filter and see when the signal exceeds a certain value, then skip some points and conserve a certain amount of the points before and after the signal

In [ ]:
length = 800
signals = np.empty((0, length)) # Empty matrix that will contain the values that we are going to save
i = 0
n_signal = 0 # Number of the signal we are detecting
trigger_level = 0.4 # Trigger level

while i < len(filt_signal):
    
    if filt_signal[i] > trigger_level:
        print("We detected a signal at the index: " + str(i))
        n_signal = n_signal + 1
        added_value = np.zeros(length)

        for j in range(0, length):
            try: 
                # Case in which the signal is ok
                added_value[j] = definitive_signal[i-100+j]
            except:
                # Case when the signal is too long
                added_value[j] = 0

        signals = np.insert(signals, 0, added_value, axis=0)

        i = i + 700
    
    else:
        i = i + 1

print("\nWe generated " + str(n_signals) + " signals with our simulation.")
print("We detected " + str(n_signal) + " signals with the trigger on the amplitude.")

t = np.linspace(0, 800, 800)
for j in range(0, n_signal):
    plt.plot(t, signals[n_signal-j-1, :])
    plt.title("Signal number " + str(j+1))
    plt.show()

We can try to do a toy-experiment to see how good is the trigger on the amplitude. This can be improved if we change some parameters of the filter process or of the trigger.

In [ ]:
#------Con questi valori ci mette circa 15 minuti--------#
n_toys = 1000
n_value = 1e5
prob = 0.0005
points = 101
approx = 5
length = 800
trigger_level = 0.4
#---------------------------------------------------------#

n_created_signals = 0
n_detected_signals = 0

# A basic approach can consist of just calculating how many signals we lost with the simple amplitude trigger

for i in range(0, n_toys):
    
    signal, n = SignalSimulation(n_value, prob)
    n_created_signals = n_created_signals + n

    filt_signal = Filter(signal, points, approx)

    n_detected = AmplitudeTrigger(signal, filt_signal, length, trigger_level)
    n_detected_signals = n_detected_signals + n_detected

perc = (n_detected_signals/n_created_signals)*100

print("\nWe detected " + str(n_detected_signals) + " signal on the " + str(n_created_signals) + " we generated.")
print("With the trigger on the amplitude we detected " + str(perc) + "% of the signals we generated (or arrived on our detector).")